In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import xarray as xr
import matplotlib.pyplot as plt

pd.options.plotting.backend = "plotly"

## Load dataset

---


In [2]:
bats_bottle = xr.open_zarr("../../2_processed/bats_bottle.zarr")
bats_primary_production = xr.open_zarr("../../2_processed/bats_primary_production.zarr")
bats_zooplankton = xr.open_zarr("../../2_processed/bats_zooplankton.zarr")

In [3]:
bats_zooplankton = bats_zooplankton[
    ["dry_weight_vol_water_ratio", "wet_weight_vol_water_ratio"]
]
bats_zooplankton

<xarray.Dataset> Size: 610kB
Dimensions:                     (time: 541, is_day: 2, depth: 7, latitude: 1,
                                 longitude: 1, sieve_size: 5)
Coordinates:
  * depth                       (depth) float64 56B 50.0 100.0 ... 300.0 350.0
  * is_day                      (is_day) bool 2B False True
  * latitude                    (latitude) float64 8B 31.6
  * longitude                   (longitude) float64 8B -64.2
  * sieve_size                  (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
  * time                        (time) datetime64[ns] 4kB 1995-05-10 ... 2022...
Data variables:
    dry_weight_vol_water_ratio  (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB dask.array<chunksize=(271, 2, 7, 1, 1, 5), meta=np.ndarray>
    wet_weight_vol_water_ratio  (time, is_day, depth, latitude, longitude, sieve_size) float64 303kB dask.array<chunksize=(271, 2, 7, 1, 1, 5), meta=np.ndarray>

In [4]:
bats_primary_production

<xarray.Dataset> Size: 260kB
Dimensions:    (time: 445, depth: 8, latitude: 1, longitude: 1)
Coordinates:
  * depth      (depth) int64 64B 50 100 150 200 250 300 350 400
  * latitude   (latitude) float64 8B 31.6
  * longitude  (longitude) float64 8B -64.2
  * time       (time) datetime64[ns] 4kB 1988-12-18 ... 2022-12-16T08:11:00
Data variables:
    dark       (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>
    lt1        (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>
    lt2        (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>
    lt3        (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>
    pp         (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>
    pres       (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>
    salt       (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>
    t0         (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>
    temp       (time, depth, latitude, longitude) float64 28kB dask.array<chunksize=(445, 8, 1, 1), meta=np.ndarray>

In [5]:
bats_bottle

<xarray.Dataset> Size: 7MB
Dimensions:                        (time: 3754, depth: 8, latitude: 1,
                                    longitude: 1)
Coordinates:
  * depth                          (depth) int64 64B 50 100 150 ... 300 350 400
  * latitude                       (latitude) float64 8B 31.6
  * longitude                      (longitude) float64 8B -64.2
  * time                           (time) datetime64[ns] 30kB 1988-10-21T04:5...
Data variables: (12/27)
    CO2                            (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    CTD_salinity                   (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    POC                            (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    PON                            (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    POP                            (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    TN                             (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    ...                             ...
    salinity                       (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    sigma_theta                    (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    silicate                       (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    synechococcus                  (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    temp                           (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>
    total_dissolved_phosphorus     (time, depth, latitude, longitude) float64 240kB dask.array<chunksize=(3754, 8, 1, 1), meta=np.ndarray>

## Manage index

---


In [6]:
times = pd.DataFrame(
    {
        "btl_time": bats_bottle.time.to_series(),
        "zpk_time": bats_zooplankton.time.to_series(),
        "pp_time": bats_primary_production.time.to_series(),
    }
)
fig = px.histogram(
    times,
    x=["btl_time", "zpk_time", "pp_time"],
    title="time distribution",
    opacity=0.5,
    log_y=True,
    nbins=30,
    barmode="overlay",
    marginal="box",
    labels={"value": "time (m)", "variable": "Datasets"},
)

fig.show()

In [7]:
depths = pd.DataFrame(
    {
        "ctd_depth": bats_bottle.depth.to_series(),
        "zpk_depth": bats_zooplankton.depth.to_series(),
        "pp_depth": bats_primary_production.depth.to_series(),
    }
)
px.box(
    depths.stack().reset_index().rename(columns={"level_1": "dataset"}),
    x="dataset",
    y="depth",
)

## Migrant and resident


## Gathering

---


# TODO : Select only the wanted variables in each dataset


In [8]:
# Temporary, need to select the temp to keep
bats_bottle = bats_bottle.drop_vars("temp")

In [9]:
final_dataset = xr.merge([bats_zooplankton, bats_primary_production, bats_bottle])
final_dataset.load()

<xarray.Dataset> Size: 16MB
Dimensions:                        (depth: 8, is_day: 2, latitude: 1,
                                    longitude: 1, sieve_size: 5, time: 4428)
Coordinates:
  * depth                          (depth) float64 64B 50.0 100.0 ... 400.0
  * is_day                         (is_day) bool 2B False True
  * latitude                       (latitude) float64 8B 31.6
  * longitude                      (longitude) float64 8B -64.2
  * sieve_size                     (sieve_size) float64 40B 0.2 0.5 1.0 2.0 5.0
  * time                           (time) datetime64[ns] 35kB 1988-10-21T04:5...
Data variables: (12/37)
    dry_weight_vol_water_ratio     (time, is_day, depth, latitude, longitude, sieve_size) float64 3MB ...
    wet_weight_vol_water_ratio     (time, is_day, depth, latitude, longitude, sieve_size) float64 3MB ...
    dark                           (time, depth, latitude, longitude) float64 283kB ...
    lt1                            (time, depth, latitude, longitude) float64 283kB ...
    lt2                            (time, depth, latitude, longitude) float64 283kB ...
    lt3                            (time, depth, latitude, longitude) float64 283kB ...
    ...                             ...
    prochlorococcus                (time, depth, latitude, longitude) float64 283kB ...
    salinity                       (time, depth, latitude, longitude) float64 283kB ...
    sigma_theta                    (time, depth, latitude, longitude) float64 283kB ...
    silicate                       (time, depth, latitude, longitude) float64 283kB ...
    synechococcus                  (time, depth, latitude, longitude) float64 283kB ...
    total_dissolved_phosphorus     (time, depth, latitude, longitude) float64 283kB ...

In [10]:
final_dataset.to_zarr("../../3_post_processed/bats_product.zarr", mode="w")

/var/folders/36/grrgsqjd14j4tf6cf5ty4ykh0000gn/T/ipykernel_24057/721788021.py:1: UserWarning:

Times can't be serialized faithfully to int64 with requested units 'days since 1995-05-10'. Serializing with units 'minutes since 1995-05-10' instead. Set encoding['dtype'] to floating point dtype to serialize with units 'days since 1995-05-10'. Set encoding['units'] to 'minutes since 1995-05-10' to silence this warning .

